# BERT

In [1]:
import pandas as pd

import transformers
from transformers import BertTokenizer

In [2]:
# in alternativa usare %%writefile per creare un file py e importare questo
import sys
sys.path.append('..')

import import_ipynb
from data_preparation import Preprocessing

importing Jupyter notebook from C:\Users\Alessia\UniProjects\HLT\src\classifiers\..\data_preparation\Preprocessing.ipynb


In [3]:
df = pd.read_csv('../../data/updated_tweets.csv')

## Preprocessing

### Normalize

In [4]:
def normalize_tweet_BERT(tweet):
    tweet = Preprocessing.remove_links_mentions(tweet)
    tweet = tweet.lower()
    tweet = Preprocessing.remove_hashtag(tweet)
    tweet = Preprocessing.remove_special_characters(tweet)
 
    tweet = Preprocessing.remove_spaces(tweet)
    tweet = Preprocessing.remove_textual_emojis(tweet)
    tweet = Preprocessing.remove_not_ASCII(tweet)

    return tweet

In [5]:
df.tweet_text = [normalize_tweet_BERT(tweet) for tweet in df.tweet_text]

In [6]:
df = Preprocessing.clean_normalized_df(df)

### Tokenize

In [7]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)           #cambiare nome bert-base-uncased?

In [8]:
'''encoded_tweets = [tokenizer.encode(sent, add_special_tokens=True) for sent in df.tweet_text] 
max_len = max([len(t) for t in encoded_tweets])
max_len'''
#la differenza è che tokenizer.encode restituisce una sequenza di interi codificati che rappresentano i token,
#includendo anche l'aggiunta di token speciali e altre operazioni di preprocessing.

'encoded_tweets = [tokenizer.encode(sent, add_special_tokens=True) for sent in df.tweet_text] \nmax_len = max([len(t) for t in encoded_tweets])\nmax_len'

In [9]:
tokens = [tokenizer.tokenize(tweet) for tweet in df.tweet_text]

### Padding

Cerchiamo la frase con più token

In [10]:
max_len = max([len(t) for t in tokens])
max_len

428

In [11]:
def lunghezza_media(tokens):
    return sum(len(array) for array in tokens) / len(tokens)

avg_len_tweet = lunghezza_media(tokens)
avg_len_tweet

32.62096424287282

In [12]:
'''for t in tokens:
    if len(t) == 428:
        print(t)'''

'for t in tokens:\n    if len(t) == 428:\n        print(t)'

In [13]:
max_length = 128

In [14]:
for i in range(len(tokens)):
    t = tokens[i][:max_length - 2]  # -2 per fare spazio ai token speciali [CLS] e [SEP]
    # Aggiungi token speciali
    t.insert(0, 'CLS')
    t.append('SEP')
    tokens[i] = t

In [15]:
# tokens = [['CLS'] + t[:max_length - 2] + ['SEP'] for t in tokens]

### Creazione della maschera di attenzione

In [16]:
attention_mask = [1] * len(tokens)  # Imposta tutti i token come attivi
padding_length = max_length - len(tokens)
tokens += ['[PAD]'] * padding_length  # Aggiungi token di padding
attention_mask += [0] * padding_length  # Imposta i token di padding come inattivi

In [17]:
attention_mask

[1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,


In [18]:
# Step 4: Conversione in interi
input_ids = [tokenizer.convert_tokens_to_ids(t) for t in tokens]

In [19]:
# Step 5: Segmentazione del testo (opzionale)
segment_ids = [0] * max_length  # Se si utilizza un singolo testo, impostare tutti i segment IDs a 0

In [20]:
pip install torch

Note: you may need to restart the kernel to use updated packages.


In [21]:
import torch

# Converti in tensori PyTorch
input_ids = torch.tensor([input_ids])
attention_mask = torch.tensor([attention_mask])
segment_ids = torch.tensor([segment_ids])

AttributeError: module 'torch' has no attribute 'tensor'